In [1]:
#pip install neo4j

In [2]:
import pandas as pd
import numpy as np
import pyodbc
from neo4j import GraphDatabase

C:\Users\linew\AppData\Local\Temp\ipykernel_17632\2500859818.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
ClimateR = pd.read_csv("2023_Cities_Climate_Risk_and_Vulnerability_Assessments_20240207.csv") #It needs the encoding otherwise it can't be read

In [4]:
ClimateR.isnull().sum()

Questionnaire                                                         0
Organization Number                                                   0
Organization Name                                                     0
City                                                                439
Country/Area                                                          0
CDP Region                                                            0
C40 City                                                              0
GCoM City                                                             0
Access                                                                0
Assessment attachment and/or direct link                             37
Confirm attachment/link provided                                      0
Boundary of assessment relative to jurisdiction boundary             12
Year of publication or approval                                      26
Factors considered in assessment                                

In [5]:
ClimateR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1370 entries, 0 to 1369
Data columns (total 20 columns):
 #   Column                                                            Non-Null Count  Dtype  
---  ------                                                            --------------  -----  
 0   Questionnaire                                                     1370 non-null   object 
 1   Organization Number                                               1370 non-null   int64  
 2   Organization Name                                                 1370 non-null   object 
 3   City                                                              931 non-null    object 
 4   Country/Area                                                      1370 non-null   object 
 5   CDP Region                                                        1370 non-null   object 
 6   C40 City                                                          1370 non-null   bool   
 7   GCoM City                        

In [6]:
len(pd.unique(ClimateR['City']))

444

In [7]:
len(pd.unique(ClimateR['Country/Area']))

79

In [8]:
len(pd.unique(ClimateR['Organization Number']))

741

In [9]:
City = pd.DataFrame(columns = ["Id", "Name", "c40", "location", "contry", "Gcom"])
City.set_index("Id", inplace = True)

In [10]:
Orgnisation = pd.DataFrame(columns = ["accountNumber", "organisationName", "CityID"])
Orgnisation.set_index("accountNumber", inplace = True)
Orgnisation

,organisationName,CityID
accountNumber,,


In [11]:
CER2017 = pd.read_csv("2017_-_Cities_Emissions_Reduction_Targets_20240207.csv")

In [12]:
Ccw2017 = pd.read_csv("2017_-_Cities_Community_Wide_Emissions.csv")

In [13]:
GHG = pd.read_csv("2016_-_Citywide_GHG_Emissions_20240207.csv")

In [14]:
CER2016 = pd.read_csv("2016_-_Cities_Emissions_Reduction_Targets_20240207.csv")

In [15]:
CER2017.head()

,Account No,Organisation,City,Country,Region,Access,C40,Reporting year,Type of target,Sector,...,Baseline emissions (metric tonnes CO2e),Percentage reduction target,Target date,Estimated business as usual absolute emissions in target year (metric tonnes CO2e),​Intensity unit (emissions per),Comment,Population,Population Year,City Location,Country Location
0,54408,Aarhus Kommune,Aarhus,Denmark,Europe,Public,NaN,2017,Absolute target,NaN,...,NaN,100.0,2030,NaN,NaN,NaN,336000.0,2017.0,"(56.168393, 10.137373)","(56.26392, 9.501785)"
1,63616,Abasan Al-Kabira Municipality,Abasan Al-Kabira,State of Palestine,South and West Asia,Public,NaN,2017,Absolute target,Buildings,...,18320.0,19.0,2020,NaN,NaN,NaN,30000.0,2015.0,"(31.323126, 34.344025)","(31.9522, 35.2332)"
2,63616,Abasan Al-Kabira Municipality,Abasan Al-Kabira,State of Palestine,South and West Asia,Public,NaN,2017,Absolute target,Transport,...,6893.0,6.0,2020,NaN,NaN,NaN,30000.0,2015.0,"(31.323126, 34.344025)","(31.9522, 35.2332)"
3,1499,Ajuntament de Barcelona,Barcelona,Spain,Europe,Public,C40,2017,Absolute target,Total,...,4053.0,18.0,2020,NaN,NaN,NaN,1608746.0,2016.0,"(41.382271, 2.177506)","(40.463667, -3.74922)"
4,1499,Ajuntament de Barcelona,Barcelona,Spain,Europe,Public,C40,2017,Absolute target,Buildings,...,2164.0,1.0,2020,NaN,NaN,"Residential, Commercial and Industrial",1608746.0,2016.0,"(41.382271, 2.177506)","(40.463667, -3.74922)"


get the country table

In [16]:
Contry = CER2017[["Country", "Region", "Country Location"]].copy()
Contry3 = Ccw2017[["Country", "Region", "Country Location"]].copy()
Contry1 = CER2016[["Country", "Country Location"]].copy()
Contry2 = GHG[["Country", "Country Location"]].copy()
Contry4 = ClimateR[["Country/Area", "CDP Region"]].copy()
Contry.columns = ["Name", "Region", "location"]
Contry3.columns = ["Name", "Region", "location"]
Contry1.columns = ["Name","location"]
Contry2.columns = ["Name","location"]
Contry4.columns = ["Name", "Region"]
Contry1["Region"] = np.NaN
Contry2["Region"] = np.NaN
Contry4["location"] = np.NaN
ContryCombine = pd.concat([Contry, Contry1, Contry2, Contry3, Contry4], ignore_index = True)
ContryCombine

,Name,Region,location
0,Denmark,Europe,"(56.26392, 9.501785)"
1,State of Palestine,South and West Asia,"(31.9522, 35.2332)"
2,State of Palestine,South and West Asia,"(31.9522, 35.2332)"
3,Spain,Europe,"(40.463667, -3.74922)"
4,Spain,Europe,"(40.463667, -3.74922)"
...,...,...,...
2467,United States of America,United States of America,NaN
2468,United States of America,United States of America,NaN
2469,United States of America,United States of America,NaN
2470,United Kingdom of Great Britain and Northern I...,Europe,NaN


In [17]:
ContryCombine["not_missing"] = ContryCombine.apply(lambda x: x.count(),axis = 1)
ContryCombine = ContryCombine.sort_values(by = "not_missing", ascending = False)

In [18]:
ContryCombine

,Name,Region,location,not_missing
0,Denmark,Europe,"(56.26392, 9.501785)",3
1082,USA,North America,"(37.09024, -95.712891)",3
1089,Italy,Europe,"(41.87194, 12.56738)",3
1088,Canada,North America,"(56.130366, -106.346771)",3
1087,Colombia,Latin America,"(4.5709, -74.2973)",3
...,...,...,...,...
1240,Mexico,Latin America,NaN,2
1239,Cameroon,Africa,NaN,2
2471,Argentina,Latin America,NaN,2
290,NaN,NaN,NaN,0


In [19]:
ContryCombine = ContryCombine.drop_duplicates(subset = ["Name"],keep = "first")

In [20]:
ContryCombine

,Name,Region,location,not_missing
0,Denmark,Europe,"(56.26392, 9.501785)",3
1082,USA,North America,"(37.09024, -95.712891)",3
1089,Italy,Europe,"(41.87194, 12.56738)",3
1088,Canada,North America,"(56.130366, -106.346771)",3
1087,Colombia,Latin America,"(4.5709, -74.2973)",3
...,...,...,...,...
1498,Czechia,Europe,NaN,2
1169,Malawi,Africa,NaN,2
1125,Democratic Republic of the Congo,Africa,NaN,2
1264,"China, Hong Kong Special Administrative Region",East Asia,NaN,2


In [21]:
ContryCombine = ContryCombine.drop(columns = ["not_missing"])
ContryCombine = ContryCombine.dropna(subset = ["Name"])
ContryCombine = ContryCombine.fillna("")
ContryCombine

,Name,Region,location
0,Denmark,Europe,"(56.26392, 9.501785)"
1082,USA,North America,"(37.09024, -95.712891)"
1089,Italy,Europe,"(41.87194, 12.56738)"
1088,Canada,North America,"(56.130366, -106.346771)"
1087,Colombia,Latin America,"(4.5709, -74.2973)"
...,...,...,...
1430,Bosnia & Herzegovina,Europe,
1498,Czechia,Europe,
1169,Malawi,Africa,
1125,Democratic Republic of the Congo,Africa,


get the city table

In [22]:
CER2016.head()

,Organisation,Account No,Country,City Short Name,C40,Reporting Year,Sector,Target boundary,Baseline year,Baseline emissions (metric tonnes CO2e),Percentage reduction target,Target date,Comment,City Location,Country Location
0,Odder Kommune,58796,Denmark,Odder Kommune,NaN,2016,Total,NaN,2010,6136.0,2.0,NaN,NaN,"(55.975718, 10.149958)","(56.26392, 9.501785)"
1,Comune di Napoli,36158,Italy,Napoli,NaN,2016,Total,NaN,2005,2913434.0,25.0,2020.0,NaN,"(40.8419943, 14.2513831)","(41.87194, 12.56738)"
2,Egedal Municipality,62855,Denmark,Egedal Municipality,NaN,2016,Total,NaN,2009,268000.0,7.0,2020.0,The municipality are increasing in terms of ci...,"(55.795045, 12.238733)","(56.26392, 9.501785)"
3,Yilan County,61753,Taiwan,Yilan,NaN,2016,Total,NaN,2009,3804493.0,35.0,2020.0,NaN,"(24.702107, 121.73775)","(23.69781, 120.960515)"
4,"City of Emeryville, CA",61790,USA,"Emeryville, CA",NaN,2016,Total,Overall community emissions,2004,178832.0,40.0,2030.0,NaN,"(37.831316, -122.285247)","(37.09024, -95.712891)"


In [23]:
City1 = ClimateR[["City", "C40 City", 'City Location', 'Country/Area', 'GCoM City', "Organization Number"]].copy()
City2 = CER2017[["City", "C40", "City Location", "Country", "Account No"]].copy()
City3 = Ccw2017[["City", "C40", "City Location", "Country", "Account number"]].copy()
City4 = GHG[["City Name", "C40", "City Location", "Country", "Account Number"]].copy()
City5 = CER2016[["City Short Name", "C40", "City Location", "Country", "Account No"]].copy()
City1.columns= ["Name", "c40", "location", "country", "Gcom", "organization_id"]
City2.columns= ["Name", "c40", "location", "country", "organization_id"]
City3.columns= ["Name", "c40", "location", "country", "organization_id"]
City4.columns= ["Name", "c40", "location", "country", "organization_id"]
City5.columns= ["Name", "c40", "location", "country", "organization_id"]
City2["Gcom"] = np.NaN
City3["Gcom"] = np.NaN
City4["Gcom"] = np.NaN
City5["Gcom"] = np.NaN
CityCombine= pd.concat([City1, City2, City3, City4, City5], ignore_index = True)
#specialCharacterNames = CityCombine[CityCombine['country'] == "Côte d'Ivoire"]['country']

#for index in specialCharacterNames.index:
 #   CityCombine['country'][index] = "Côte d\'Ivoire"


#CityCombine['country'][96]
CityCombine

,Name,c40,location,country,Gcom,organization_id
0,NaN,False,NaN,Brazil,1.0,840926
1,Shenzhen,True,POINT (113.813 22.9175),China,0.0,51075
2,NaN,False,NaN,Chile,0.0,863190
3,NaN,False,NaN,Peru,1.0,930366
4,Trelleborg,False,POINT (13.1569 55.3751),Sweden,1.0,60236
...,...,...,...,...,...,...
2467,Salaberry-de-Valleyfield,NaN,"(45.250091, -74.133086)",Canada,NaN,60680
2468,Moscow,C40,"(55.755826, 37.6173)",Russia,NaN,31174
2469,Stadt Heidelberg,C40,"(49.398752, 8.672434)",Germany,NaN,31165
2470,"Davis, CA",NaN,"(38.544907, -121.740517)",USA,NaN,59552


In [24]:
CityCombine.loc[CityCombine["c40"] == "C40", "c40"] = True

In [25]:
CityCombine

,Name,c40,location,country,Gcom,organization_id
0,NaN,False,NaN,Brazil,1.0,840926
1,Shenzhen,True,POINT (113.813 22.9175),China,0.0,51075
2,NaN,False,NaN,Chile,0.0,863190
3,NaN,False,NaN,Peru,1.0,930366
4,Trelleborg,False,POINT (13.1569 55.3751),Sweden,1.0,60236
...,...,...,...,...,...,...
2467,Salaberry-de-Valleyfield,NaN,"(45.250091, -74.133086)",Canada,NaN,60680
2468,Moscow,True,"(55.755826, 37.6173)",Russia,NaN,31174
2469,Stadt Heidelberg,True,"(49.398752, 8.672434)",Germany,NaN,31165
2470,"Davis, CA",NaN,"(38.544907, -121.740517)",USA,NaN,59552


In [26]:
CityCombine.loc[pd.isna(CityCombine["c40"]), "c40"] = False
CityCombine

,Name,c40,location,country,Gcom,organization_id
0,NaN,False,NaN,Brazil,1.0,840926
1,Shenzhen,True,POINT (113.813 22.9175),China,0.0,51075
2,NaN,False,NaN,Chile,0.0,863190
3,NaN,False,NaN,Peru,1.0,930366
4,Trelleborg,False,POINT (13.1569 55.3751),Sweden,1.0,60236
...,...,...,...,...,...,...
2467,Salaberry-de-Valleyfield,False,"(45.250091, -74.133086)",Canada,NaN,60680
2468,Moscow,True,"(55.755826, 37.6173)",Russia,NaN,31174
2469,Stadt Heidelberg,True,"(49.398752, 8.672434)",Germany,NaN,31165
2470,"Davis, CA",False,"(38.544907, -121.740517)",USA,NaN,59552


In [27]:
CityCombine2 = CityCombine
#CityCombine2 = CityCombine2.fillna("")
CityCombine2["c40"] = CityCombine2["c40"].replace({True:1,False:0})


CityCombine2["Name"] = CityCombine2["Name"].fillna("")
CityCombine2["location"] = CityCombine2["location"].fillna("")
CityCombine2["country"] = CityCombine2["country"].fillna("")
CityCombine2["Gcom"] = CityCombine2["Gcom"].astype("Int64")

CityCombine2["Gcom"] = CityCombine2["Gcom"].fillna(-1).astype("Int64")

CityCombine2
#CityCombine2["c40"].isnull().values.any()
#CityCombine2["country"].isnull().values.any() ##checkes for no values.

C:\Users\linew\AppData\Local\Temp\ipykernel_17632\605724142.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  CityCombine2["c40"] = CityCombine2["c40"].replace({True:1,False:0})


,Name,c40,location,country,Gcom,organization_id
0,,0,,Brazil,1,840926
1,Shenzhen,1,POINT (113.813 22.9175),China,0,51075
2,,0,,Chile,0,863190
3,,0,,Peru,1,930366
4,Trelleborg,0,POINT (13.1569 55.3751),Sweden,1,60236
...,...,...,...,...,...,...
2467,Salaberry-de-Valleyfield,0,"(45.250091, -74.133086)",Canada,-1,60680
2468,Moscow,1,"(55.755826, 37.6173)",Russia,-1,31174
2469,Stadt Heidelberg,1,"(49.398752, 8.672434)",Germany,-1,31165
2470,"Davis, CA",0,"(38.544907, -121.740517)",USA,-1,59552


<h1>Get organization table

In [28]:
Org1 = ClimateR[["Organization Number", "Organization Name"]].copy()
Org2 = CER2017[["Account No", "Organisation"]].copy()
Org3 = Ccw2017[["Account number", "Organization"]].copy()
Org4 = GHG[["Account Number", "City Name"]].copy()
Org5 = CER2016[["Account No", "Organisation"]].copy()
Org1.columns= ["Account No", "Organization Name"]
Org2.columns= ["Account No", "Organization Name"]
Org3.columns= ["Account No", "Organization Name"]
Org4.columns= ["Account No", "Organization Name"]
Org5.columns= ["Account No", "Organization Name"]
OrgCombine= pd.concat([Org1, Org2, Org3, Org4, Org5], ignore_index = True)
OrgCombine["city_id"] = 0
OrgCombine


# Index difference between MSSQL and Pandas Dataframe

CityLinkOrg = CityCombine2['organization_id']
CityLinkOrg

for index in CityLinkOrg.index:
    OrgCombine.loc[OrgCombine["Account No"] == CityLinkOrg[index], 'city_id'] = index+1

OrgCombine

,Account No,Organization Name,city_id
0,840926,Prefeitura de Serra Talhada,1
1,51075,City of Shenzhen,1587
2,863190,Renca,1340
3,930366,Municipalidad Distrital de Yura,4
4,60236,Trelleborg Municipality,1293
...,...,...,...
2467,60680,Ville de Salaberry-de-Valleyfield,2468
2468,31174,Moscow Government,2469
2469,31165,Stadt Heidelberg,2470
2470,59552,"City of Davis, CA",2471


In [29]:
OrgCombine = OrgCombine.drop_duplicates(subset = ["Account No"],keep = "first")
OrgCombine

,Account No,Organization Name,city_id
0,840926,Prefeitura de Serra Talhada,1
1,51075,City of Shenzhen,1587
2,863190,Renca,1340
3,930366,Municipalidad Distrital de Yura,4
4,60236,Trelleborg Municipality,1293
...,...,...,...
2301,55467,"City of Aurora, CO",2302
2421,43934,City of Perth,2422
2422,44295,Gwangyang City,2423
2423,51105,Hongcheon County,2424


get Gas tabel

In [30]:
GHG["Gases included"]

0                            CO2; CH4; N2O
1      CO2; PFCs; CH4; SF6; N2O; NF3; HFCs
2                            CO2; CH4; N2O
3                            CO2; CH4; N2O
4                            CO2; CH4; N2O
                      ...                 
182                          CO2; CH4; N2O
183                          CO2; CH4; N2O
184                               CO2; CH4
185                               CO2; CH4
186                          CO2; CH4; N2O
Name: Gases included, Length: 187, dtype: object

In [31]:
list(CER2016.columns)

['Organisation',
 'Account No',
 'Country',
 'City Short Name',
 'C40',
 'Reporting Year',
 'Sector',
 'Target boundary',
 'Baseline year',
 'Baseline emissions (metric tonnes CO2e)',
 'Percentage reduction target',
 'Target date',
 'Comment',
 'City Location',
 'Country Location']

In [32]:
gas1 = GHG[["Gases included"]].copy()
gas2 = Ccw2017[["Gases included"]].copy()
gas1.columns= ['Gases_included']
gas2.columns= ['Gases_included']
gasCombine= pd.concat([gas1, gas2], ignore_index = True)

In [33]:
gas = gasCombine.Gases_included.str.split(';').apply(pd.Series)
gas.index = gasCombine.set_index(['Gases_included']).index
gas = gas.stack().reset_index(['Gases_included'])
gas.drop(['Gases_included'], axis=1, inplace=True)
gas = gas.drop_duplicates(subset = [0],keep = "first")
gas = gas.reset_index()
gas.drop(['index'], axis=1, inplace=True)
gas.columns = ['Gas']
gas['Gas'] = gas['Gas'].str.strip()
gas

,Gas
0,CO2
1,CH4
2,N2O
3,PFCs
4,SF6
5,NF3
6,HFCs
7,CH4


current poplulation

In [34]:
CP1 = ClimateR[["Population", "Population Year"]].copy()
CP1.columns = ["Population", "Year"]
CP1.dropna(subset=['Year'], inplace=True)
CP1['Year'] = CP1['Year'].astype(int)

CP1

,Population,Year
0,92228,2023
1,17662000,2022
2,162517,2022
3,33346,2017
4,46649,2022
...,...,...
1365,44480,2020
1366,118403,2020
1367,737015,2020
1368,522100,2021


<h1>GDP Currency

In [35]:
cu1 = GHG[["GDP Currency"]].copy()
cu2 = Ccw2017[["GDP Currency"]].copy()
cu1.columns = ["GDP_Currency"]
cu2.columns = ["GDP_Currency"]
cuCombine= pd.concat([cu1, cu2], ignore_index = True)
cuCombine = cuCombine.drop_duplicates(subset = ["GDP_Currency"],keep = "first")
cuCombine = cuCombine.dropna()
cuCombine = cuCombine.reset_index()
cuCombine.drop(['index'], axis=1, inplace=True)
cuCombine

,GDP_Currency
0,CAD Canadian Dollar
1,USD US Dollar
2,KRW South Korean Won
3,AUD Australian Dollar
4,GBP Pound Sterling
5,BRL Brazilian Real
6,CHF Swiss Franc
7,IDR Rupiah
8,BOB Boliviano
9,PHP Philippine Peso


In [36]:
cu = cuCombine.GDP_Currency.str.split(" ", n=1).apply(pd.Series)
cu.columns = ["Id", "Name"]
cu

,Id,Name
0,CAD,Canadian Dollar
1,USD,US Dollar
2,KRW,South Korean Won
3,AUD,Australian Dollar
4,GBP,Pound Sterling
5,BRL,Brazilian Real
6,CHF,Swiss Franc
7,IDR,Rupiah
8,BOB,Boliviano
9,PHP,Philippine Peso


<h1>get GDP table</h1>

In [37]:
list(Ccw2017.columns)

['Account number',
 'Organization',
 'City',
 'Country',
 'Region',
 'C40',
 'Access',
 'Reporting year',
 'Accounting year',
 'Boundary',
 'Protocol',
 'Protocol column',
 'Gases included',
 'Total emissions (metric tonnes CO2e)',
 'Scopes Included ',
 'Total Scope 1 Emissions (metric tonnes CO2e)',
 'Total Scope 2 Emissions (metric tonnes CO2e)',
 'Comment',
 'Increase/Decrease from last year',
 'Reason for increase/decrease in emissions',
 'Population',
 'Population year',
 'GDP',
 'GDP Currency',
 'GDP Year',
 'GDP Source',
 'Average annual temperature (in Celsius)\u200b',
 '\u200bAverage altitude (m)',
 '\u200bLand area (in square km)',
 'City Location',
 'Country Location']

In [38]:
GDP1 = GHG[["City GDP", "GDP Currency", "Year of GDP", "GDP Source"]].copy()
GDP2 = Ccw2017[["GDP", "GDP Currency", "GDP Year", "GDP Source"]].copy()
GDP1.columns= ["City", "Currency", "Year", "Source"]
GDP2.columns= ["City", "Currency", "Year", "Source"]
GDPCombine = pd.concat([GDP1, GDP2], ignore_index = True)
GDPCombine

,City,Currency,Year,Source
0,1.201180e+11,CAD Canadian Dollar,2014.0,"Ville de Montréal, Montréal en statistiques, h..."
1,8.777300e+10,NaN,2013.0,UK ONS (using 1.56 USD/GBP) Figures provided i...
2,1.000000e+10,USD US Dollar,2014.0,Bureau of Economic Analysis10.4
3,2.357000e+11,USD US Dollar,2014.0,http://www.bea.gov/iTable/iTable.cfm?reqid=70&...
4,1.540000e+10,CAD Canadian Dollar,2014.0,"Conference Board of Canada, London Census Metr..."
...,...,...,...,...
411,NaN,NaN,NaN,NaN
412,4.676285e+09,BRL Brazilian Real,2014.0,IBGE
413,6.450000e+10,EUR Euro,2016.0,https://public.tableau.com/profile/gemeente.am...
414,NaN,USD US Dollar,NaN,City GDP data is not available


In [39]:
GDP = GDPCombine.Currency.str.split(" ", n=1).apply(pd.Series)
GDP

,0,1
0,CAD,Canadian Dollar
1,NaN,NaN
2,USD,US Dollar
3,USD,US Dollar
4,CAD,Canadian Dollar
...,...,...
411,NaN,NaN
412,BRL,Brazilian Real
413,EUR,Euro
414,USD,US Dollar


In [40]:
GDP = pd.merge(GDPCombine, GDP, left_index=True, right_index=True)
GDP= GDP.drop(["Currency", 1], axis = 'columns')
GDP.columns = ["City", "Year", "Source", "Currency"]
GDP

,City,Year,Source,Currency
0,1.201180e+11,2014.0,"Ville de Montréal, Montréal en statistiques, h...",CAD
1,8.777300e+10,2013.0,UK ONS (using 1.56 USD/GBP) Figures provided i...,NaN
2,1.000000e+10,2014.0,Bureau of Economic Analysis10.4,USD
3,2.357000e+11,2014.0,http://www.bea.gov/iTable/iTable.cfm?reqid=70&...,USD
4,1.540000e+10,2014.0,"Conference Board of Canada, London Census Metr...",CAD
...,...,...,...,...
411,NaN,NaN,NaN,NaN
412,4.676285e+09,2014.0,IBGE,BRL
413,6.450000e+10,2016.0,https://public.tableau.com/profile/gemeente.am...,EUR
414,NaN,NaN,City GDP data is not available,USD


<h1>City Reductions</h1>

In [41]:
CER1 = CER2016[["Account No", "Reporting Year", "Sector", "Target boundary", "Baseline year", "Baseline emissions (metric tonnes CO2e)", "Percentage reduction target", "Target date", "Comment"]].copy()
CER2 = CER2017[["Account No", "Reporting year", "Sector", "Type of target", "Baseline year", "Baseline emissions (metric tonnes CO2e)", "Percentage reduction target", "Target date", "Comment",'Population','Population Year', "Estimated business as usual absolute emissions in target year (metric tonnes CO2e)", "\u200bIntensity unit (emissions per)"]].copy()
CER1.columns = ["Account No", "Reporting_Year", "Sector", "Target_boundary", "Baseline_year", "Baseline_emissions", "Percentage_reduction_target", "Target_date", "Comment"]
CER2.columns = ["Account No", "Reporting_Year", "Sector", "Target_boundary", "Baseline_year", "Baseline_emissions", "Percentage_reduction_target", "Target_date", "Comment", 'Population','Population Year',"Estimated_business_as_usual_absolute_emissions_in_target_year", "Intensity unit"]
CER1["Intensity unit"] = np.NaN
CER1["Estimated_business_as_usual_absolute_emissions_in_target_year"] = np.NaN
CER1['Population'] = np.NaN
CER1['Population Year'] = np.NaN
CERCombine = pd.concat([CER1, CER2], ignore_index = True)
CERCombine

,Account No,Reporting_Year,Sector,Target_boundary,Baseline_year,Baseline_emissions,Percentage_reduction_target,Target_date,Comment,Intensity unit,Estimated_business_as_usual_absolute_emissions_in_target_year,Population,Population Year
0,58796,2016,Total,NaN,2010,6136.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,36158,2016,Total,NaN,2005,2913434.0,25.0,2020.0,NaN,NaN,NaN,NaN,NaN
2,62855,2016,Total,NaN,2009,268000.0,7.0,2020.0,The municipality are increasing in terms of ci...,NaN,NaN,NaN,NaN
3,61753,2016,Total,NaN,2009,3804493.0,35.0,2020.0,NaN,NaN,NaN,NaN,NaN
4,61790,2016,Total,Overall community emissions,2004,178832.0,40.0,2030.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,43937,2017,Total,Absolute target,2001,1105654.0,40.0,2030,NaN,NaN,NaN,209102.0,2017.0
682,43937,2017,Total,Absolute target,2001,1105654.0,65.0,2040,NaN,NaN,NaN,209102.0,2017.0
683,43937,2017,Total,Absolute target,2001,1105654.0,80.0,2050,NaN,NaN,NaN,209102.0,2017.0
684,43937,2017,Other: Corporate,Absolute target,2014,92832.0,80.0,2050,NaN,NaN,NaN,209102.0,2017.0


<h1>city wide GHG</h1>

In [42]:
list(GHG.columns)

['Account Number',
 'City Name',
 'Country',
 'City Short Name',
 'C40',
 'Reporting Year',
 'Measurement Year',
 'Boundary',
 'Primary Methodology',
 'Methodology Details',
 'Gases included',
 'Total City-wide Emissions (metric tonnes CO2e)',
 'Total Scope 1 Emissions (metric tonnes CO2e)',
 'Total Scope 2 Emissions (metric tonnes CO2e)',
 'Increase/Decrease from last year',
 'Reason for increase/decrease in emissions',
 'Current Population Year',
 'Current Population',
 'City GDP',
 'GDP Currency',
 'Year of GDP',
 'GDP Source',
 'Average annual temperature (in Celsius)\u200b',
 '\u200bLand area (in square km)',
 'Average altitude (m)',
 'City Location',
 'Country Location']

In [43]:
GHG1 = GHG[["Account Number", "Reporting Year", "Boundary", "Primary Methodology", "Total City-wide Emissions (metric tonnes CO2e)", "Total Scope 1 Emissions (metric tonnes CO2e)", "Total Scope 2 Emissions (metric tonnes CO2e)", "Increase/Decrease from last year",'Current Population Year','Current Population', 'Average annual temperature (in Celsius)\u200b','\u200bLand area (in square km)', 'Average altitude (m)']].copy()
GHG1.columns = ["Account_Number", "Reporting_Year", "Boundry", "Primary_Methodology", "Total_City-wide_Emissions", "Total_Scope_1_Emissions", "Total_Scope_2_Emissions", "Increase/Decrease_from_last year", 'Current Population Year', 'Current Population','Average_annual_temperature','Land_area', 'Average_altitude']
GHG1

,Account_Number,Reporting_Year,Boundry,Primary_Methodology,Total_City-wide_Emissions,Total_Scope_1_Emissions,Total_Scope_2_Emissions,Increase/Decrease_from_last year,Current Population Year,Current Population,Average_annual_temperature,Land_area,Average_altitude
0,35894,2016,Other: The regional entity that constitutes th...,2006 IPCC Guidelines for National Greenhouse G...,13722942.0,13634331.0,88611.0,Stayed the same,2011,1886481.0,6.8,500.0,17.0
1,35898,2016,A metropolitan area,Global Protocol for Community-Scale Greenhouse...,14889318.0,NaN,5302028.0,Decreased,2014,2762000.0,9.5,1277.0,250.0
2,54128,2016,Administrative boundary of a local government,Global Protocol for Community-Scale Greenhouse...,4437665.0,NaN,NaN,This is our first year of calculation,2014,236995.0,10.4,288.0,1373.0
3,35879,2016,Administrative boundary of a local government,U.S. Community Protocol for Accounting and Rep...,4794708.0,2890572.0,1904136.0,Increased,2014,407207.0,7.9,139.8,264.0
4,50558,2016,Administrative boundary of a local government,International Emissions Analysis Protocol (ICLEI),3070000.0,2771000.0,299000.0,Increased,2014,375000.0,7.9,421.0,251.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,59165,2016,A metropolitan area,Global Protocol for Community-Scale Greenhouse...,362078.0,9924.0,214496.0,Decreased,2014,66656.0,NaN,24.9,NaN
183,31108,2016,Administrative boundary of a local government,U.S. Community Protocol for Accounting and Rep...,33428301.0,16075539.0,14511262.0,This is our first year of calculation,2013,2195914.0,21.0,1625.0,13.0
184,31167,2016,A metropolitan area,Global Protocol for Community-Scale Greenhouse...,29426266.0,29426266.0,NaN,Other,2016,21000000.0,30.0,3577.0,34.0
185,54088,2016,A metropolitan area,International Emissions Analysis Protocol (ICLEI),283061.0,NaN,NaN,NaN,2011,78698.0,12.1,63.8,195.0


<h1>City community wide Emmision</h1>

In [44]:
list(Ccw2017.columns)

['Account number',
 'Organization',
 'City',
 'Country',
 'Region',
 'C40',
 'Access',
 'Reporting year',
 'Accounting year',
 'Boundary',
 'Protocol',
 'Protocol column',
 'Gases included',
 'Total emissions (metric tonnes CO2e)',
 'Scopes Included ',
 'Total Scope 1 Emissions (metric tonnes CO2e)',
 'Total Scope 2 Emissions (metric tonnes CO2e)',
 'Comment',
 'Increase/Decrease from last year',
 'Reason for increase/decrease in emissions',
 'Population',
 'Population year',
 'GDP',
 'GDP Currency',
 'GDP Year',
 'GDP Source',
 'Average annual temperature (in Celsius)\u200b',
 '\u200bAverage altitude (m)',
 '\u200bLand area (in square km)',
 'City Location',
 'Country Location']

In [45]:
CCW1 = Ccw2017[['Account number', 'Access','Reporting year','Accounting year','Boundary','Protocol','Protocol column','Gases included','Total emissions (metric tonnes CO2e)','Scopes Included ','Total Scope 1 Emissions (metric tonnes CO2e)','Total Scope 2 Emissions (metric tonnes CO2e)','Comment','Increase/Decrease from last year','Reason for increase/decrease in emissions','Population','Population year','Average annual temperature (in Celsius)\u200b','\u200bAverage altitude (m)','\u200bLand area (in square km)']].copy()
CCW1.columns = ['Account_number', 'Access','Reporting_year','Accounting_year','Boundary','Protocol','Protocol_column','Gases_included','Total_emissions','Scopes_Included ','Total_Scope_1_Emissions','Total_Scope_2_Emissions','Comment','Increase/Decrease_from_last_year','Reason_for_increase/decrease_in_emissions','Population','Population year','Average_annual_temperature','Average_altitude','Land_area']
CCW1

,Account_number,Access,Reporting_year,Accounting_year,Boundary,Protocol,Protocol_column,Gases_included,Total_emissions,Scopes_Included,Total_Scope_1_Emissions,Total_Scope_2_Emissions,Comment,Increase/Decrease_from_last_year,Reason_for_increase/decrease_in_emissions,Population,Population year,Average_annual_temperature,Average_altitude,Land_area
0,49363,Public,2017,2013-07-01 - 2014-06-30,A metropolitan area,Global Protocol for Community-Scale Greenhouse...,NaN,NaN,12232310.0,"Total Scope 1, Scope 2 and Waste Scope 3 (Tota...",2249502.00,9982808.00,NaN,Increased,We are gradually improving the accuracy of our...,1152115,2011,17.5,69.0,1950.0
1,31171,Public,2017,2014-01-01 - 2014-12-31,Administrative boundary of a local government,Global Protocol for Community-Scale Greenhouse...,Adaptation from Madrid Air Quality Inventory e...,NaN,9236196.0,"Total Scope 1, Scope 2 and Waste Scope 3 (Tota...",7069783.00,3408424.00,NaN,Decreased,"A decrease of a 2,17 % in global emissions in...",3165883,2016,15.0,655.0,604.0
2,3417,Public,2017,2015-01-01 - 2015-12-31,Administrative boundary of a local government,Global Protocol for Community-Scale Greenhouse...,ConEdison (ConEd) provided data on use of city...,NaN,52042186.0,"Total Scope 1, Scope 2 and Waste Scope 3 (Tota...",38962264.00,14343743.00,NaN,Decreased,The most significant driver was a milder winte...,8537673,2016,13.0,10.0,784.0
3,59537,Public,2017,2015-01-01 - 2015-12-31,Administrative boundary of a local government,U.S. Community Protocol for Accounting and Rep...,"Since 2006, the City has utilized software dev...",CO2; CH4; N2O,1604007.0,Total Scope 1 and Scope 2,898681.00,759270.00,The one sector of tracking and reporting accur...,Decreased,Overall Community GHGs have fallen 1% since 20...,131044,2015,18.2,192.0,227.8
4,35894,Public,2017,2013-01-01 - 2013-12-31,A metropolitan area,Global Protocol for Community-Scale Greenhouse...,NaN,NaN,10197555.0,"Total Scope 1, Scope 2 and Waste Scope 3 (Tota...",10729801.00,61912.00,NaN,Decreased,Agglomeration-wide GHG emissions were 25% lowe...,1959987,2013,6.8,17.0,500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,19233,Public,2017,2014-01-01 - 2014-12-31,Administrative boundary of a local government,International Emissions Analysis Protocol (ICLEI),ICLEI - CCP Protocol (Cities for Climate Prote...,CO2,275646.0,Total Scope 1 and Scope 2,NaN,NaN,NaN,Decreased,The city has done a emission inventory in 2009...,79465,2011,15.0,75.0,407.0
225,60273,Public,2017,2014-01-01 - 2014-12-31,Administrative boundary of a local government,Global Protocol for Community-Scale Greenhouse...,Para o primeiro inventário de emissões de gase...,NaN,131931.0,"Total Scope 1, Scope 2 and Waste Scope 3 (Tota...",115719.73,16201.19,NaN,This is our first year of calculation,As emissões reportadas nesta plataforma são pr...,33729,2016,17.7,973.0,243.1
226,31148,Public,2017,2015-12-01 - 2016-12-31,Administrative boundary of a local government,Other,"Local standard based on electricity, gas, dist...",CO2; N2O,4138000.0,Total Scope 1 and Scope 2,NaN,2460000.00,"Figures of Electricity, gas and district heat ...",Decreased,The decrease is very strong (10%) The reason f...,834713,2016,10.9,2.0,219.0
227,54119,Public,2017,2016-01-01 - 2016-12-31,Administrative boundary of a local government,U.S. Community Protocol for Accounting and Rep...,TCR/LGOP,CO2; CH4; N2O,153948.0,Total Scope 1 and Scope 2,153948.00,NaN,High for utilities (data from municipally owne...,Decreased,Combined City Municipal Operations and Palo Al...,66478,2017,15.0,7.0,67.0


In [46]:
CCW = CCW1.Accounting_year.str.split("-", n=3).apply(pd.Series)
CCW["Accounting_year_Start"] = CCW[0].astype(str)+"-"+CCW[1].astype(str)+"-"+CCW[2].astype(str)
CCW = CCW.drop([0, 1, 2], axis= "columns")
CCW1 = CCW1.drop(["Accounting_year"], axis ="columns")
CCW.columns =  ["Accounting_year_end", "Accounting_year"]
CCW = pd.merge(CCW1, CCW, left_index=True, right_index=True)
CCW

,Account_number,Access,Reporting_year,Boundary,Protocol,Protocol_column,Gases_included,Total_emissions,Scopes_Included,Total_Scope_1_Emissions,...,Comment,Increase/Decrease_from_last_year,Reason_for_increase/decrease_in_emissions,Population,Population year,Average_annual_temperature,Average_altitude,Land_area,Accounting_year_end,Accounting_year
0,49363,Public,2017,A metropolitan area,Global Protocol for Community-Scale Greenhouse...,NaN,NaN,12232310.0,"Total Scope 1, Scope 2 and Waste Scope 3 (Tota...",2249502.00,...,NaN,Increased,We are gradually improving the accuracy of our...,1152115,2011,17.5,69.0,1950.0,2014-06-30,2013-07-01
1,31171,Public,2017,Administrative boundary of a local government,Global Protocol for Community-Scale Greenhouse...,Adaptation from Madrid Air Quality Inventory e...,NaN,9236196.0,"Total Scope 1, Scope 2 and Waste Scope 3 (Tota...",7069783.00,...,NaN,Decreased,"A decrease of a 2,17 % in global emissions in...",3165883,2016,15.0,655.0,604.0,2014-12-31,2014-01-01
2,3417,Public,2017,Administrative boundary of a local government,Global Protocol for Community-Scale Greenhouse...,ConEdison (ConEd) provided data on use of city...,NaN,52042186.0,"Total Scope 1, Scope 2 and Waste Scope 3 (Tota...",38962264.00,...,NaN,Decreased,The most significant driver was a milder winte...,8537673,2016,13.0,10.0,784.0,2015-12-31,2015-01-01
3,59537,Public,2017,Administrative boundary of a local government,U.S. Community Protocol for Accounting and Rep...,"Since 2006, the City has utilized software dev...",CO2; CH4; N2O,1604007.0,Total Scope 1 and Scope 2,898681.00,...,The one sector of tracking and reporting accur...,Decreased,Overall Community GHGs have fallen 1% since 20...,131044,2015,18.2,192.0,227.8,2015-12-31,2015-01-01
4,35894,Public,2017,A metropolitan area,Global Protocol for Community-Scale Greenhouse...,NaN,NaN,10197555.0,"Total Scope 1, Scope 2 and Waste Scope 3 (Tota...",10729801.00,...,NaN,Decreased,Agglomeration-wide GHG emissions were 25% lowe...,1959987,2013,6.8,17.0,500.0,2013-12-31,2013-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,19233,Public,2017,Administrative boundary of a local government,International Emissions Analysis Protocol (ICLEI),ICLEI - CCP Protocol (Cities for Climate Prote...,CO2,275646.0,Total Scope 1 and Scope 2,NaN,...,NaN,Decreased,The city has done a emission inventory in 2009...,79465,2011,15.0,75.0,407.0,2014-12-31,2014-01-01
225,60273,Public,2017,Administrative boundary of a local government,Global Protocol for Community-Scale Greenhouse...,Para o primeiro inventário de emissões de gase...,NaN,131931.0,"Total Scope 1, Scope 2 and Waste Scope 3 (Tota...",115719.73,...,NaN,This is our first year of calculation,As emissões reportadas nesta plataforma são pr...,33729,2016,17.7,973.0,243.1,2014-12-31,2014-01-01
226,31148,Public,2017,Administrative boundary of a local government,Other,"Local standard based on electricity, gas, dist...",CO2; N2O,4138000.0,Total Scope 1 and Scope 2,NaN,...,"Figures of Electricity, gas and district heat ...",Decreased,The decrease is very strong (10%) The reason f...,834713,2016,10.9,2.0,219.0,2016-12-31,2015-12-01
227,54119,Public,2017,Administrative boundary of a local government,U.S. Community Protocol for Accounting and Rep...,TCR/LGOP,CO2; CH4; N2O,153948.0,Total Scope 1 and Scope 2,153948.00,...,High for utilities (data from municipally owne...,Decreased,Combined City Municipal Operations and Palo Al...,66478,2017,15.0,7.0,67.0,2016-12-31,2016-01-01


<h1>Cites climate risk and vu</h1>

In [47]:
list(ClimateR.columns)

['Questionnaire',
 'Organization Number',
 'Organization Name',
 'City',
 'Country/Area',
 'CDP Region',
 'C40 City',
 'GCoM City',
 'Access',
 'Assessment attachment and/or direct link',
 'Confirm attachment/link provided',
 'Boundary of assessment relative to jurisdiction boundary',
 'Year of publication or approval',
 'Factors considered in assessment',
 'Primary author(s) of assessment',
 'Does the city have adaptation goal(s) and/or an adaptation plan?',
 'Population',
 'Population Year',
 'City Location',
 'Last update']

In [48]:
CLi1 = ClimateR[['Questionnaire','Organization Number','Access','Assessment attachment and/or direct link','Confirm attachment/link provided','Boundary of assessment relative to jurisdiction boundary','Year of publication or approval','Factors considered in assessment','Primary author(s) of assessment','Does the city have adaptation goal(s) and/or an adaptation plan?', 'Population','Population Year']].copy()
CLi1.columns = ['Questionnaire','Organization_Number','Access','Assessment_attachment_and/or_direct_link','Confirm_attachment/link_provided','Boundary_of_assessment_relative_to_jurisdiction_boundary','Year_of_publication_or_approval','Factors_considered_in_assessment','Primary_author(s)_of_assessment','Does_the_city_have_adaptation_goal(s)_and/or_an_adaptation_plan?','Population','Population Year']
CLi1

,Questionnaire,Organization_Number,Access,Assessment_attachment_and/or_direct_link,Confirm_attachment/link_provided,Boundary_of_assessment_relative_to_jurisdiction_boundary,Year_of_publication_or_approval,Factors_considered_in_assessment,Primary_author(s)_of_assessment,Does_the_city_have_adaptation_goal(s)_and/or_an_adaptation_plan?,Population,Population Year
0,Cities 2023,840926,public,https://drive.google.com/file/d/19DMxxK532IQSL...,The assessment can be accessed (unrestricted) ...,Same - covers entire jurisdiction and nothing ...,2022.0,Assessment considers vulnerable populations; A...,Dedicated team within jurisdiction; Relevant d...,Adaptation goal(s) and adaptation plan,92228,2023.0
1,Cities 2023,51075,public,NaN,The assessment can be accessed (unrestricted) ...,Same - covers entire jurisdiction and nothing ...,2022.0,Assessment considers nature; Assessment consid...,Consultant; Dedicated team within jurisdiction...,Adaptation plan,17662000,2022.0
2,Cities 2023,863190,public,PLCC_Renca (08 Abril).pdf,The assessment has been attached,Same - covers entire jurisdiction and nothing ...,2019.0,A process has been established for prioritizin...,Question not applicable,Adaptation goal(s) and adaptation plan,162517,2022.0
3,Cities 2023,930366,public,https://sigrid.cenepred.gob.pe/sigridv3/docume...,The assessment can be accessed (unrestricted) ...,Same - covers entire jurisdiction and nothing ...,2022.0,Identified hazards have been incorporated into...,Dedicated team within jurisdiction,Incomplete report,33346,2017.0
4,Cities 2023,60236,public,https://moten.trelleborg.se/welcome-sv/namnder...,The assessment can be accessed (unrestricted) ...,Same - covers entire jurisdiction and nothing ...,2018.0,A process has been established for prioritizin...,Dedicated team within jurisdiction,Adaptation goal(s) and adaptation plan,46649,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1365,Cities 2023,58871,public,https://www.salemma.gov/sites/g/files/vyhlif79...,The assessment can be accessed (unrestricted) ...,Same - covers entire jurisdiction and nothing ...,2021.0,A process has been established to update the a...,Consultant,Adaptation goal(s) and adaptation plan,44480,2020.0
1366,Cities 2023,55800,public,ccpralewifepreparednessplan_cambridge.pdf,The assessment has been attached and can be ac...,Smaller - covers only part of the jurisdiction...,2017.0,Assessment considers nature; Assessment consid...,Dedicated team within jurisdiction; Relevant d...,Adaptation goal(s) and adaptation plan,118403,2020.0
1367,Cities 2023,16581,public,https://your.kingcounty.gov/dnrp/climate/docum...,The assessment can be accessed (unrestricted) ...,Larger - covers the whole jurisdiction and adj...,2021.0,Assessment considers nature; Assessment consid...,Regional/ state/ provincial government,Adaptation goal(s) and adaptation plan,737015,2020.0
1368,Cities 2023,826450,public,Final PHSMT Climate and Health in County Durha...,The assessment has been attached,Same - covers entire jurisdiction and nothing ...,2023.0,A process has been established for prioritizin...,Dedicated team within jurisdiction; Relevant d...,Adaptation plan,522100,2021.0


In [49]:
OrgCombine_reset = OrgCombine.reset_index(drop=True)
reportYears = [2016, 2017, 2023]
ClimateR = pd.read_csv("2023_Cities_Climate_Risk_and_Vulnerability_Assessments_20240207.csv") 
ClimateR_reset = ClimateR.reset_index(drop=True)
CountryCombine_reset = ContryCombine.reset_index(drop=True)

In [59]:
# Connecting to the Neo4j graph database
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "abcd1234")

def createOrganisation(tx, org_acc_no, org_name, org_city_id):
    query = tx.run("""
        CREATE (o:Organisation {account_no: $acc_no, name: $name, city_id: $city_id})
    """, acc_no = org_acc_no, name = org_name, city_id=org_city_id)

def addReportYears(tx, year):
    query = tx.run("""
        CREATE (y:Year {year: $year})
    """, year=year)

def insertContryCombine(tx, row):
    query = tx.run("""
        CREATE (c:Country {name: $name, region: $region, location: $location})
    """, name=row["Name"], region = row["Region"], location = row["location"])
    
def insertClimateR(tx, row):
    query = tx.run("""
        MATCH (y:Year {year: 2023})
        MATCH (O:Organisation {account_no: $acc_no})
        MATCH (C:Country {name: $country_name})
        CREATE (r:Report {cdp_region: $cdp_region, c40_city: $c40_city, gcom_city: $gcom_city, access: $access, assessment_attachment_and_or_direct_link: $assessment_attachment_and_or_direct_link, confirm_attachment_link_provided: $confirm_attachment_link_provided, boundary_of_assessment_relative_to_jurisdiction_boundary: $boundary_of_assessment_relative_to_jurisdiction_boundary, year_of_publication_or_approval: $year_of_publication_or_approval, factors_considered_in_assessment: $factors_considered_in_assessment, primary_authors_of_assessment: $primary_authors_of_assessment, does_the_city_have_adaptation_goals_and_or_an_adaptation_plan: $does_the_city_have_adaptation_goals_and_or_an_adaptation_plan, population: $population, population_year: $population_year, city_location: $city_location, last_update: $last_update})
        MERGE (r)-[:REPORT_YEAR]->(y)
        MERGE (r)-[:FROM_ORGANISATION]->(O)
        MERGE (r)-[:COUNTRY_ORIGIN]->(C)
    """,acc_no=row["Organization Number"], country_name=row["Country/Area"], cdp_region=row["CDP Region"], c40_city=row["C40 City"], gcom_city=row["GCoM City"], access=row["Access"], assessment_attachment_and_or_direct_link=row["Assessment attachment and/or direct link"], confirm_attachment_link_provided=row["Confirm attachment/link provided"], boundary_of_assessment_relative_to_jurisdiction_boundary=row["Boundary of assessment relative to jurisdiction boundary"], year_of_publication_or_approval=row["Year of publication or approval"], factors_considered_in_assessment=row["Factors considered in assessment"], primary_authors_of_assessment=row["Primary author(s) of assessment"], does_the_city_have_adaptation_goals_and_or_an_adaptation_plan=row["Does the city have adaptation goal(s) and/or an adaptation plan?"], population=row["Population"], population_year=row["Population Year"], city_location=row["City Location"], last_update=row["Last update"])

def insertCcw(tx, row):
    query = tx.run("""
        MATCH (y:Year {year: 2017})
        MATCH (O:Organisation {account_no: $acc_no})
        MATCH (C:Country {name: $country_name})
        CREATE (r:Report {c40: $c40, access: $access, reporting_year: $reporting_year, accounting_year: $accounting_year, boundary: $boundary, protocol: $protocol, protocol_column: $protocol_column, gases_included: $gases_included, total_emissions_metric_tonnes_co2e: $total_emissions_metric_tonnes_co2e, scopes_included_: $scopes_included_, total_scope_1_emissions_metric_tonnes_co2e: $total_scope_1_emissions_metric_tonnes_co2e, total_scope_2_emissions_metric_tonnes_co2e: $total_scope_2_emissions_metric_tonnes_co2e, comment: $comment, increase_decrease_from_last_year: $increase_decrease_from_last_year, reason_for_increase_decrease_in_emissions: $reason_for_increase_decrease_in_emissions, population: $population, population_year: $population_year, gdp: $gdp, gdp_currency: $gdp_currency, gdp_year: $gdp_year, gdp_source: $gdp_source, average_annual_temperature_in_celsius: $average_annual_temperature_in_celsius, average_altitude_m: $average_altitude_m, land_area_in_square_km: $land_area_in_square_km, city_location: $city_location, country_location: $country_location})
        MERGE (r)-[:REPORT_YEAR]->(y)
        MERGE (r)-[:FROM_ORGANISATION]->(O)
        MERGE (r)-[:COUNTRY_ORIGIN]->(C)
        """,acc_no=row["Account number"], country_name=row["Country"], c40=row["C40"], access=row["Access"], reporting_year=row["Reporting year"], accounting_year=row["Accounting year"], boundary=row["Boundary"], protocol=row["Protocol"], protocol_column=row["Protocol column"], gases_included=row["Gases included"], total_emissions_metric_tonnes_co2e=row["Total emissions (metric tonnes CO2e)"], scopes_included_=row["Scopes Included "], total_scope_1_emissions_metric_tonnes_co2e=row["Total Scope 1 Emissions (metric tonnes CO2e)"], total_scope_2_emissions_metric_tonnes_co2e=row["Total Scope 2 Emissions (metric tonnes CO2e)"], comment=row["Comment"], increase_decrease_from_last_year=row["Increase/Decrease from last year"], reason_for_increase_decrease_in_emissions=row["Reason for increase/decrease in emissions"], population=row["Population"], population_year=row["Population year"], gdp=row["GDP"], gdp_currency=row["GDP Currency"], gdp_year=row["GDP Year"], gdp_source=row["GDP Source"], average_annual_temperature_in_celsius=row["Average annual temperature (in Celsius)\u200b"], average_altitude_m=row["\u200bAverage altitude (m)"], land_area_in_square_km=row["\u200bLand area (in square km)"], city_location=row["City Location"], country_location=row["Country Location"])

def insertGHG(tx, row):
    query = tx.run("""
        MATCH (y:Year {year: 2016})
        MATCH (O:Organisation {account_no: $acc_no})
        MATCH (C:Country {name: $country_name})
        CREATE (r: Report {city_name: $city_name, city_short_name: $city_name, c40: $c40, reporting_year: $reporting_year, measurement_year: $measurement_year, boundary: $boundary, primary_methodology: $primary_methodology, methodology_details: $methodology_details, gases_included: $gases_included, total_city_wide_emissions_metric_tonnes_co2e: $total_city_wide_emissions_metric_tonnes_co2e, total_scope_1_emissions_metric_tonnes_co2e: $total_scope_1_emissions_metric_tonnes_co2e, total_scope_2_emissions_metric_tonnes_co2e: $total_scope_2_emissions_metric_tonnes_co2e, increase_decrease_from_last_year: $increase_decrease_from_last_year, reason_for_increase_decrease_in_emissions: $reason_for_increase_decrease_in_emissions, current_population_year: $current_population_year, current_population: $current_population, city_gdp: $city_gdp, gdp_currency: $gdp_currency, year_of_gdp: $year_of_gdp, gdp_source: $gdp_source, average_annual_temperature_in_celsiusb: $average_annual_temperature_in_celsius, land_area_in_square_km: $land_area_in_square_km, average_altitude_m: $average_altitude_m, city_location: $city_location, country_location: $country_location})
        MERGE (r)-[:REPORT_YEAR]->(y)
        MERGE (r)-[:FROM_ORGANISATION]->(O)
        MERGE (r)-[:COUNTRY_ORIGIN]->(C)
        """, acc_no=row["Account Number"], country_name=row["Country"], city_name=row["City Name"], city_short_name=row["City Short Name"], c40=row["C40"], reporting_year=row["Reporting Year"], measurement_year=row["Measurement Year"], boundary=row["Boundary"], primary_methodology=row["Primary Methodology"],  methodology_details=row["Methodology Details"], gases_included=row["Gases included"], total_city_wide_emissions_metric_tonnes_co2e=row["Total City-wide Emissions (metric tonnes CO2e)"], total_scope_1_emissions_metric_tonnes_co2e=row["Total Scope 1 Emissions (metric tonnes CO2e)"], total_scope_2_emissions_metric_tonnes_co2e=row["Total Scope 2 Emissions (metric tonnes CO2e)"], increase_decrease_from_last_year=row["Increase/Decrease from last year"], reason_for_increase_decrease_in_emissions=row["Reason for increase/decrease in emissions"], current_population_year=row["Current Population Year"], current_population=row["Current Population"], city_gdp=row["City GDP"], gdp_currency=row["GDP Currency"], year_of_gdp=row["Year of GDP"], gdp_source=row["GDP Source"], average_annual_temperature_in_celsius=row["Average annual temperature (in Celsius)\u200b"], land_area_in_square_km=row["\u200bLand area (in square km)"], average_altitude_m=row["Average altitude (m)"], city_location=row["City Location"], country_location=row["Country Location"])

def insertCER2016(tx, row):
    query = tx.run("""
        MATCH (y:Year {year: $year})
        MATCH (O:Organisation {account_no: $acc_no})
        MATCH (C:Country {name: $country_name})
        CREATE (r: Report {city_short_name: $city_short_name, c40: $c40, sector: $sector, target_boundary: $target_boundary, baseline_year: $baseline_year, baseline_emissions_metric_tonnes_co2e: $baseline_emissions_metric_tonnes_co2e, percentage_reduction_target: $percentage_reduction_target, target_date: $target_date, comment: $comment, city_location: $city_location, country_location: $country_location})
        MERGE (r)-[:REPORT_YEAR]->(y)
        MERGE (r)-[:FROM_ORGANISATION]->(O)
        MERGE (r)-[:COUNTRY_ORIGIN]->(C)
        """, acc_no=row["Account No"], country_name=row["Country"], city_short_name=row["City Short Name"], c40=row["C40"], year=row["Reporting Year"], sector=row["Sector"], target_boundary=row["Target boundary"], baseline_year=row["Baseline year"], baseline_emissions_metric_tonnes_co2e=row["Baseline emissions (metric tonnes CO2e)"], percentage_reduction_target=row["Percentage reduction target"], target_date=row["Target date"], comment=row["Comment"], city_location=row["City Location"], country_location=row["Country Location"])


def insertCER2017(tx, row):
    query = tx.run("""
        MATCH (y:Year {year: 2017})
        MATCH (O:Organisation {account_no: $acc_no})
        MATCH (C:Country {name: $country_name})
        CREATE (r: Report {city: $city, region: $region, access: $access, c40: $c40, reporting_year: $reporting_year, type_of_target: $type_of_target, sector: $sector, baseline_year: $baseline_year, baseline_emissions_metric_tonnes_co2e: $baseline_emissions_metric_tonnes_co2e, percentage_reduction_target: $percentage_reduction_target, target_date: $target_date, estimated_business_as_usual_absolute_emissions_in_target_year_metric_tonnes_co2e: $estimated_business_as_usual_absolute_emissions_in_target_year_metric_tonnes_co2e, intensity_unit_emissions_per: $intensity_unit_emissions_per, comment: $comment, population: $population, population_year: $population_year, city_location: $city_location, country_location: $country_location})
        MERGE (r)-[:REPORT_YEAR]->(y)
        MERGE (r)-[:FROM_ORGANISATION]->(O)
        MERGE (r)-[:COUNTRY_ORIGIN]->(C)
        """,acc_no=row["Account No"], city=row["City"], country_name=row["Country"], region=row["Region"], access=row["Access"], c40=row["C40"], reporting_year=row["Reporting year"], type_of_target=row["Type of target"], sector=row["Sector"], baseline_year=row["Baseline year"], baseline_emissions_metric_tonnes_co2e=row["Baseline emissions (metric tonnes CO2e)"], percentage_reduction_target=row["Percentage reduction target"], target_date=row["Target date"], estimated_business_as_usual_absolute_emissions_in_target_year_metric_tonnes_co2e=row["Estimated business as usual absolute emissions in target year (metric tonnes CO2e)"], intensity_unit_emissions_per=row["\u200bIntensity unit (emissions per)"], comment=row["Comment"], population=row["Population"], population_year=row["Population Year"], city_location=row["City Location"], country_location=row["Country Location"])

        


In [52]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    with driver.session(database="dbmain") as session:
        
         # Adding organisations to the DB
        for i in range(len(OrgCombine_reset)):
            session.execute_write(createOrganisation, OrgCombine_reset.loc[i, "Account No"], OrgCombine_reset.loc[i, "Organization Name"], OrgCombine_reset.loc[i, "city_id"])
            
        # Adding report years to the DB
        for year in reportYears:
            session.execute_write(addReportYears, year)

        # Adding Countries
        for i in range(len(CountryCombine_reset)):
            session.execute_write(insertContryCombine, CountryCombine_reset.loc[i])

In [53]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    with driver.session(database="dbmain") as session:
        
# Adding ClimateR reports to the DB 
        for i in range (len(ClimateR_reset)):
            session.execute_write(insertClimateR, ClimateR_reset.loc[i])
            

In [54]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    with driver.session(database="dbmain") as session:
#Adding City community wide Emmision to the DB
        for i in range (len(Ccw2017)):
            session.execute_write(insertCcw, Ccw2017.loc[i])

In [55]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    with driver.session(database="dbmain") as session:
#Adding GHG to the DB
        for i in range (len(GHG)):
            session.execute_write(insertGHG, GHG.loc[i])


In [57]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    with driver.session(database="dbmain") as session:
        #Adding CER2016 to the DB
        for i in range (len(CER2016)):
            session.execute_write(insertCER2016, CER2016.loc[i])


In [60]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    with driver.session(database="dbmain") as session:
        
        #Adding CER2017 to the DB
        for i in range (len(CER2017)):
            session.execute_write(insertCER2017, CER2017.loc[i])

In [ ]:
list(CER2017.columns)

In [ ]:
list(CER2016.columns)

In [ ]:
result = ", ".join(f"{item.lower().replace(' ', '_').replace('(', '').replace(')', '')}: ${item.lower().replace(' ', '_').replace('(', '').replace(')', '')}" for item in list(CER2017.columns))

In [ ]:
result = ", ".join(f'{item.lower().replace(" ", "_").replace("(", "").replace(")", "")}=row["{item}"]'for item in list(CER2017.columns))

In [ ]:
result